This notebook will attempt to debug the time issues we are currently facing. There are two issues here:
1. That MOM6 is not accepting GREGORIAN as a calendar.
2. That there is some strange thing happening with the timestep of a particular forcing file. 

We'll start with the second issue first. For this, I want to pull in the times for all forcing files for (1) `sharkbay_new` - an RYF experiment that we know works, and (2) `sharkbay_iaf` the experiment that is currently having the issue.

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import glob
import os
from dask.distributed import Client
client = Client(threads_per_worker=1)
client


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 8
Total threads: 8,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36603,Workers: 8
Dashboard: /proxy/8787/status,Total threads: 8
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:36379,Total threads: 1
Dashboard: /proxy/42473/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:44705,


In [6]:
[path for path in IAF_files if 'rlds' in path]

['/scratch/v45/jr5971/mom6_regional_configs/sharkbay_iaf/jra_forcing/rlds_input4MIPs_atmosphericState_OMIP_MRI-JRA55-do-1-5-0_gr_197001010000-197012312100.nc']

In [27]:
keyword = 'prsn'
ds = xr.open_dataset([path for path in IAF_files if keyword in path][0])
ds = ds.assign_coords(time=rlds.time - pd.Timedelta(hours=1.5))
os.remove([path for path in IAF_files if keyword in path][0])
ds.to_netcdf([path for path in IAF_files if keyword in path][0], 
             encoding={'time':{'calendar':'gregorian'}})


In [22]:
ds = rlds.assign_coords(time=rlds.time - pd.Timedelta(hours=1.5))


In [15]:
import os

In [24]:
[path for path in IAF_files if 'rsds' in path][0]

'/scratch/v45/jr5971/mom6_regional_configs/sharkbay_iaf/jra_forcing/rsds_input4MIPs_atmosphericState_OMIP_MRI-JRA55-do-1-5-0_gr_197001010130-197012312230.nc'

In [23]:
os.remove([path for path in IAF_files if 'rsds' in path][0])
ds.to_netcdf([path for path in IAF_files if 'rsds' in path][0], 
             encoding={'time':{'calendar':'gregorian'}})

In [19]:
ds['time']

<xarray.DataArray 'time' (time: 2920)>
array(['1970-01-01T00:00:00.000000000', '1970-01-01T03:00:00.000000000',
       '1970-01-01T06:00:00.000000000', ..., '1970-12-31T15:00:00.000000000',
       '1970-12-31T18:00:00.000000000', '1970-12-31T21:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 1970-01-01 ... 1970-12-31T21:00:00

In [8]:
rlds.indexes

Indexes:
    time     DatetimeIndex(['1970-01-01 01:30:00', '1970-01-01 04:30:00',
               '1970-01-01 07:30:00', '1970-01-01 10:30:00',
               '1970-01-01 13:30:00', '1970-01-01 16:30:00',
               '1970-01-01 19:30:00', '1970-01-01 22:30:00',
               '1970-01-02 01:30:00', '1970-01-02 04:30:00',
               ...
               '1970-12-30 19:30:00', '1970-12-30 22:30:00',
               '1970-12-31 01:30:00', '1970-12-31 04:30:00',
               '1970-12-31 07:30:00', '1970-12-31 10:30:00',
               '1970-12-31 13:30:00', '1970-12-31 16:30:00',
               '1970-12-31 19:30:00', '1970-12-31 22:30:00'],
              dtype='datetime64[ns]', name='time', length=2920, freq=None)
    lat      Float64Index([-89.5700895506066,  -89.013176131022,  -88.452973836713,
              -87.8920284453444, -87.3308011797376, -86.7694375145276,
              -86.2079976214231, -85.6465108479528, -85.0849932009119,
              -84.5234541489144,
              

In [10]:
tas = xr.open_dataset([path for path in IAF_files if 'tas' in path][0])
tas

<xarray.Dataset>
Dimensions:    (time: 2920, bnds: 2, lat: 320, lon: 640)
Coordinates:
  * time       (time) datetime64[ns] 1970-01-01 ... 1970-12-31T21:00:00
  * lat        (lat) float64 -89.57 -89.01 -88.45 -87.89 ... 88.45 89.01 89.57
  * lon        (lon) float64 0.0 0.5625 1.125 1.688 ... 357.8 358.3 358.9 359.4
    height     float64 ...
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] ...
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    tas        (time, lat, lon) float32 ...
Attributes: (12/36)
    Conventions:         CF-1.7 CMIP-6.2
    activity_id:         input4MIPs
    cell_measures:       area: areacella
    comment:             Based on JRA-55 reanalysis (1958-01 to 2020-07)
    contact:             Hiroyuki Tsujino (htsujino@mri-jma.go.jp)
    creation_date:       2020-09-15T18:09:30Z
    ...                  ...
    target_mip:          OMIP
    title:               MRI JRA55-do 1.5.0 dataset prepared for input4MIPs
    tracking_id:         hdl:21.14100/b2c221a7-6e22-4539-a88f-88565ed44655
    variable_id:         tas
    license:             OMIP boundary condition data produced by MRI is lice...
    cmor_version:        3.6.0

In [11]:
tas.indexes

Indexes:
    time     DatetimeIndex(['1970-01-01 00:00:00', '1970-01-01 03:00:00',
               '1970-01-01 06:00:00', '1970-01-01 09:00:00',
               '1970-01-01 12:00:00', '1970-01-01 15:00:00',
               '1970-01-01 18:00:00', '1970-01-01 21:00:00',
               '1970-01-02 00:00:00', '1970-01-02 03:00:00',
               ...
               '1970-12-30 18:00:00', '1970-12-30 21:00:00',
               '1970-12-31 00:00:00', '1970-12-31 03:00:00',
               '1970-12-31 06:00:00', '1970-12-31 09:00:00',
               '1970-12-31 12:00:00', '1970-12-31 15:00:00',
               '1970-12-31 18:00:00', '1970-12-31 21:00:00'],
              dtype='datetime64[ns]', name='time', length=2920, freq=None)
    lat      Float64Index([-89.5700895506066,  -89.013176131022,  -88.452973836713,
              -87.8920284453444, -87.3308011797376, -86.7694375145276,
              -86.2079976214231, -85.6465108479528, -85.0849932009119,
              -84.5234541489144,
              

In [2]:
# path to ryf files for sharkbay_new
RYF_files = glob.glob("/g/data/ik11/inputs/JRA-55/RYF/v1-3/*1990*.nc")
IAF_files = glob.glob("/scratch/v45/jr5971/mom6_regional_configs/sharkbay_iaf/jra_forcing/*.nc")

# pull in all files into a dataset...
ryf_ds = xr.open_mfdataset(RYF_files)
iaf_ds = xr.open_mfdataset(IAF_files)

/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.10/lib/python3.9/site-packages/xarray/core/indexing.py:1378: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  value = value[(slice(None),) * axis + (subkey,)]
/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.10/lib/python3.9/site-packages/xarray/core/indexing.py:1378: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_c

KeyboardInterrupt: 

Process Dask Worker process (from Nanny):
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.10/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.10/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.10/lib/python3.9/site-packages/distributed/process.py", line 190, in _run
    target(*args, **kwargs)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.10/lib/python3.9/site-packages/distributed/nanny.py", line 986, in _run
    asyncio.run(run())
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.10/lib/python3.9/asyncio/runners.py", line 47, in run
    _cancel_all_tasks(loop)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.10/lib/python3.9/asyncio/runners.py", line 63, in _cancel_all_tasks
    loop.run_unt

Now to fix the following error:
`period of list exceeds modulo period,file=INPUT/forcing/forcing_obc_segment_001.nc,field=u_segment_001`

In [7]:
import os

In [8]:
%%time
forcing = "/scratch/v45/jr5971/mom6_regional_configs/sharkbay_iaf/forcing"

for seg in range(1,5):
    ds = xr.open_dataset(f"{forcing}/forcing_obc_segment_00{seg}.nc")
    ds['time'].attrs.pop('modulo')
    os.remove(f"{forcing}/forcing_obc_segment_00{seg}.nc")
    ds.to_netcdf(f"{forcing}/forcing_obc_segment_00{seg}.nc")



CPU times: user 2min 36s, sys: 1min 41s, total: 4min 18s
Wall time: 4min 11s


In [3]:
segtime.attrs.pop('modulo')

' '

In [4]:
segtime.attrs

{}